Pre-Requisites for this notebook:
* Install Anaconda & install redshift_connector with conda
* Either launch Jupyter Notebook from Anaconda Navigator or install and setup VS Code with Python and Jupyter Extensions
* Connect to VPN
* Get your RedShift credentials

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import redshift_connector
from datetime import datetime
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [2]:
# Import user credentials from user_credentials.csv file
usr_cred = pd.read_csv('user_credentials.csv')
username = usr_cred.iloc[0]['Username']
passwd = usr_cred.iloc[0]['Password']
opnaikey = usr_cred.iloc[0]['OpenAI Key']

User credentials are entered onto a local csv file in the format shown below. ***Don't add your credentials to this file, or share your user credentials csv!***
| Username    | Password | OpenAI Key |
|-------------|----------|------------|
| rv-employee | 1234     | 567#       |

In [3]:
# Connect to RedShift and pull data
# Redshift connection details
host = 'dbops-redshift-cluster-apple.cd92olv6lp21.us-east-1.redshift.amazonaws.com'
port = '5439'
database = 'dev'
user = username # Use your username
password = passwd # Use your password
# Select query 1
query_1 = """
select
eventdate,
sum(spend) as spend,
sum(spend)/nullif(sum((clicks)),0) as CPC,
sum(clicks)/nullif(sum((impressions)),0) as CTR,
sum(offeraccepted_total) as accounts,
sum(offeraccepted_total)/nullif(sum(clicks),0) as CTA,
sum(total_apply_now_clickers)/nullif(sum(clicks),0) as CTAN,
sum(initiated_total)/nullif(sum(total_apply_now_clickers),0) as initiaterate,
sum(submitted_total)/nullif(sum(initiated_total),0) as submitrate,
sum(offeraccepted_total)/nullif(sum(approved_total),0) as acceptancerate,
sum(approved_total)/nullif(sum(submitted_total),0) as approvalrate,
(sum(spend)/nullif(sum(offeraccepted_total),0))*1.17 as GrossCPA,
sum(impressions)/nullif(avg((searchimpressionshare)),0) as UOI
from apple_reporting.applecard_paidsearch_view
where eventdate between (current_date - 3) and (current_date -2)
and budgettype ilike 'Performance'
group by 1
order by 1 DESC;
"""
try:
    # Connect to Redshift
    conn = redshift_connector.connect(
        host = host,
        port = port,
        database = database,
        user = user,
        password = password
    )
    # Create a cursor
    cursor = conn.cursor()
    # Execute the select query
    cursor.execute(query_1)
    # Fetch all the results
    results_1 = cursor.fetchall()
    # Close the cursor and connection
    cursor.close()
    conn.close()
except redshift_connector.Error as e:
    print('Error connecting to Redshift:', e)

In [4]:
# Store data in dataframe and add column names
df_performance = pd.DataFrame(results_1)
df_performance.columns = ['Date', 'Spend', 'CPC', 'CTR', 'Accounts', 'CTA', 'CTAN', 'Initiate Rate', 'Submit Rate', 'Acceptance Rate', 'Approval Rate',\
              'Gross CPA', 'UOI']

In [5]:
# Define some functions
def curr_form(amount):
    return "${:,.2f}".format(amount)

def percent_form(amount):
    return "{:.0%}".format(amount)

def change_dod(sum_array):
    return (sum_array.iloc[0]/sum_array.iloc[1] - 1)

In [6]:
# Generate weekly report top bullet
total_spend = df_performance['Spend']
total_accounts = df_performance['Accounts']
total_cpa = df_performance['Gross CPA']

total_cpc = df_performance['CPC']
total_ctr = df_performance['CTR']
total_uoi = df_performance['UOI']

total_ctan = df_performance['CTAN']
total_apr = df_performance['Approval Rate']
total_acr = df_performance['Acceptance Rate']
total_ir = df_performance['Initiate Rate']
total_sr = df_performance['Submit Rate']

In [7]:
# Spend drivers
cpc_change = change_dod(total_cpc)
ctr_change = change_dod(total_ctr)
uoi_change = change_dod(total_uoi)
spend_change = change_dod(total_spend)

# Account drivers
ctan_change = change_dod(total_ctan)
apr_change = change_dod(total_apr)
acr_change = change_dod(total_acr)
ir_change = change_dod(total_ir)
sr_change = change_dod(total_sr)
account_change = change_dod(total_accounts)

In [8]:
# Output biggest spend driver
x = float('inf')
y = []
if spend_change < 0:  # Assuming spend_volume_change is your DoD change of spend volume
    if x > cpc_change:
        x = cpc_change
        y = 'CPC'
    if x > ctr_change:
        x = ctr_change
        y = 'CTR'
    if x > uoi_change:
        x = uoi_change
        y = 'UOI'
else:  # spend_volume_change is positive
    x = float('-inf')  # Reset x to negative infinity
    if x < cpc_change:
        x = cpc_change
        y = 'CPC'
    if x < ctr_change:
        x = ctr_change
        y = 'CTR'
    if x < uoi_change:
        x = uoi_change
        y = 'UOI'

print(y)

UOI


In [9]:
# Output biggest account driver
a = float('inf')
b = []
if account_change < 0:  # Assuming account_change is your DoD change of account volume
    if a > ctan_change:
        a = ctan_change
        b = 'CTAN'
    if a > apr_change:
        a = apr_change
        b = 'Approval Rate'
    if a > acr_change:
        a = acr_change
        b = 'Acceptance Rate'
    if a > ir_change:
        a = ir_change
        b = 'Initiate Rate'
    if a > sr_change:
        a = sr_change
        b = 'Submit Rate'
    if a > ctr_change:
        a = ctr_change
        b = 'CTR'
    if a > uoi_change:
        a = uoi_change
        b = 'UOI'
else:  # account_change is positive
    a = float('-inf')  # Reset a to negative infinity
    if a < ctan_change:
        a = ctan_change
        b = 'CTAN'
    if a < apr_change:
        a = apr_change
        b = 'Approval Rate'
    if a < acr_change:
        a = acr_change
        b = 'Acceptance Rate'
    if a < ir_change:
        a = ir_change
        b = 'Initiate Rate'
    if a < sr_change:
        a = sr_change
        b = 'Submit Rate'
    if a < ctr_change:
        a = ctr_change
        b = 'CTR'
    if a < uoi_change:
        a = uoi_change
        b = 'UOI'

print(b)

Approval Rate


In [10]:
top_bullet = "Search spent {} ({} DoD) driving {} accounts ({} DoD) and yielding a {} CPA ({} DoD). The main driver for spend is {} ({} DoD). The main driver for accounts is {} ({} DoD).".format(\
    curr_form(total_spend.iloc[0]), percent_form(change_dod(total_spend)), round(total_accounts[0]), percent_form(change_dod(total_accounts)),
        curr_form(total_cpa[0]), percent_form(change_dod(total_cpa)), y, percent_form(x), b, percent_form(a))
print(top_bullet)

Search spent $28,491.08 (2% DoD) driving 242 accounts (-5% DoD) and yielding a $137.75 CPA (8% DoD). The main driver for spend is UOI (13% DoD). The main driver for accounts is Approval Rate (-9% DoD).
